In [23]:
drive.mount('/content/drive')
!pip install -q tensorflow tensorflowjs matplotlib
import os, tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
tf.get_logger().setLevel('ERROR')
BASE_DIR = '/content/drive/MyDrive'
DATASET_PATH = f'{BASE_DIR}/dataset'
MODEL_PATH   = f'{BASE_DIR}/plant_model.keras'
TFJS_PATH    = f'{BASE_DIR}/plant_model_js'
os.makedirs(TFJS_PATH, exist_ok=True)
print(f"Các đường dẫn đã được thiết lập. Sẵn sàng cho Cell 2.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Các đường dẫn đã được thiết lập. Sẵn sàng cho Cell 2.


In [24]:

IMG_SIZE, BATCH, EPOCHS_NEW, EPOCHS_FINE = 256, 32, 10, 1

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
    zoom_range=0.2, horizontal_flip=True
)

train_gen = datagen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=BATCH, class_mode='categorical',
                                        subset='training', shuffle=True)
val_gen = datagen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                      batch_size=BATCH, class_mode='categorical',
                                      subset='validation', shuffle=False)

num_classes = len(train_gen.class_indices)

Found 7674 images belonging to 5 classes.
Found 1917 images belonging to 5 classes.


In [25]:
def build_model(num_classes):
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(base.input, out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

if os.path.exists(MODEL_PATH):
    print(f"Đang tải model từ: {MODEL_PATH}")
    try:
        model = load_model(MODEL_PATH)

        if model.input_shape[1] != IMG_SIZE:
            print(f"Lỗi: Shape của model đã lưu {model.input_shape} không khớp với IMG_SIZE={IMG_SIZE}.")
            print("Sẽ build model mới...")
            raise ValueError("Input shape mismatch")

        print("Model đã tải. Cài đặt fine-tuning...")
        for layer in model.layers[-20:]:
            if not isinstance(layer, Dropout): layer.trainable = True
        epochs = EPOCHS_FINE

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                      loss='categorical_crossentropy', metrics=['accuracy'])

    except Exception as e:
        print(f"Không thể load model (lỗi: {e}). Building model mới.")
        model = build_model(num_classes)
        epochs = EPOCHS_NEW
else:
    print("Không tìm thấy model. Building model mới.")
    model = build_model(num_classes)
    epochs = EPOCHS_NEW

callbacks = [
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=epochs,
                    callbacks=callbacks)

Đang tải model từ: /content/drive/MyDrive/plant_model.keras
Lỗi: Shape của model đã lưu (None, 224, 224, 3) không khớp với IMG_SIZE=256.
Sẽ build model mới...
Không thể load model (lỗi: Input shape mismatch). Building model mới.


/tmp/ipython-input-3279960038.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 1530s 6s/step - accuracy: 0.7991 - loss: 0.5782 - val_accuracy: 0.9770 - val_loss: 0.0740
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 672s 3s/step - accuracy: 0.9637 - loss: 0.1075 - val_accuracy: 0.9833 - val_loss: 0.0546
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 658s 3s/step - accuracy: 0.9720 - loss: 0.0837 - val_accuracy: 0.9812 - val_loss: 0.0527
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 656s 3s/step - accuracy: 0.9753 - loss: 0.0708 - val_accuracy: 0.9838 - val_loss: 0.0410
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 654s 3s/step - accuracy: 0.9804 - loss: 0.0636 - val_accuracy: 0.9870 - val_loss: 0.0415
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 661s 3s/step - accuracy: 0.9772 - loss: 0.0688 - val_accuracy: 0.9854 - val_loss: 0.0368
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 658s 3s/step - accuracy: 0.9789 - loss: 0.0607 - val_accuracy: 0.9812 - val_loss: 0.0532
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 665s 3s/step - accuracy: 0.9847 - loss: 0.0427 - val_acc

In [26]:
import tensorflowjs as tfjs
model = load_model(MODEL_PATH)
!rm -rf {TFJS_PATH}
os.makedirs(TFJS_PATH, exist_ok=True)

print(f"Đang chuyển đổi model sang TF.js tại: {TFJS_PATH}")
tfjs.converters.save_keras_model(model, TFJS_PATH)

labels = list(train_gen.class_indices.keys())
with open(os.path.join(TFJS_PATH, 'labels.js'), 'w') as f:
    f.write(f"const CLASS_NAMES = {labels};")

print(f"Đã lưu model và file labels.js thành công.")
print(f"MODEL: {MODEL_PATH}")
print(f"TFJS:  {TFJS_PATH}")
print(f"Labels ({len(labels)} classes): {labels}")

Đang chuyển đổi model sang TF.js tại: /content/drive/MyDrive/plant_model_js
failed to lookup keras version from the file,
    this is likely a weight only file
Đã lưu model và file labels.js thành công.
MODEL: /content/drive/MyDrive/plant_model.keras
TFJS:  /content/drive/MyDrive/plant_model_js
Labels (5 classes): ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy']
